# Importações


In [177]:
import urllib
from itertools import product
from os import getenv
from sqlalchemy import create_engine
from dotenv import load_dotenv

import numpy as np
import pandas as pd
from datetime import datetime, timedelta, time
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.express as px
from fuzzywuzzy import process
from enum import Enum


FERIADOS = pd.read_csv("../assets/feriados.csv")

### Types


In [178]:
# cSpell: disable
class BSColorsEnum(Enum):

    DANGER_COLOR = "#dc3545"

    WARNING_COLOR = "#ffc107"

    SUCCESS_COLOR = "#198754"

    GREY_500_COLOR = "#adb5bd"

    GREY_600_COLOR = "#6c757d"

    GREY_700_COLOR = "#495057"

    GREY_800_COLOR = "#343a40"

    GREY_900_COLOR = "#212529"

    PRIMARY_COLOR = "#0d6efd"

    SECONDARY_COLOR = "#6c757d"

    INFO_COLOR = "#0dcaf0"

    GRAY_COLOR = "#adb5bd"

    TEAL_COLOR = "#20c997"

    ORANGE_COLOR = "#fd7e14"

    INDIGO_COLOR = "#6610f2"

    PINK_COLOR = "#d63384"

    PURPLE_COLOR = "#6f42c1"

    GREY_400_COLOR = "#ced4da"

    SPACE_CADET_COLOR = "#282f44"

    BLUE_DELFT_COLOR = "#0d6efd"


class IndicatorType(Enum):
    PERFORMANCE = "performance"
    REPAIR = "reparo"
    EFFICIENCY = "eficiencia"


def get_color(value, max_value):
    """
    Retorna uma cor hexadecimal com base no valor fornecido e no valor máximo.

    Parâmetros:
    value (float): O valor para o qual a cor será calculada.
    max_value (float): O valor máximo possível.

    Retorna:
    str: Uma cor hexadecimal correspondente ao valor fornecido.
    """

    # Cria um mapa de cores que vai do vermelho ao verde
    cmap = plt.get_cmap("RdYlGn")

    # Normaliza o valor para um número entre 0 e 1
    normalized_value = float(value) / max_value

    # Obtém a cor correspondente do mapa de cores
    rgba_color = cmap(normalized_value)

    # Converte a cor RGBA para uma string de cor hexadecimal
    hex_color = (
        f"#{int(rgba_color[0]*255):02x}{int(rgba_color[1]*255):02x}{int(rgba_color[2]*255):02x}"
    )

    return hex_color

# Database


## Conexão com o banco de dados


In [179]:
# database/connection.py

# cSpell: disable=invalid-name
load_dotenv()


class Connection:
    """
    Class Connection
    """

    def __init__(self):
        """
        Constructor

        Args:
            user (str): user
            password (str): password
            database (str): database
            driver (str): driver
            server (str): server

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        self.__user = getenv("PYMSSQL_USER")
        self.__password = getenv("PYMSSQL_PASSWORD")
        self.__database = getenv("PYMSSQL_DATABASE_AUTOMACAO")
        self.__database_totvsdb = getenv("PYMSSQL_DATABASE_TOTVSDB")
        self.__driver = "{ODBC Driver 17 for SQL Server}"
        self.__server = getenv("PYMSSQL_SERVER")

    def get_connection_automacao(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            # pylint: disable=consider-using-f-string
            conexao_automacao = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
            return conexao_automacao
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

    def get_connection_totvsdb(self):
        """
        Get connection

        Returns:
            object: connection

        Usage:
            >>> from connection import Connection
            >>> connection = Connection()
            >>> connection.get_connection()
        """
        try:
            params = urllib.parse.quote_plus(
                f"DRIVER={self.__driver};"
                f"SERVER={self.__server};"
                f"DATABASE={self.__database_totvsdb};"
                f"UID={self.__user};"
                f"PWD={self.__password};"
            )
            conexao_totvsdb = create_engine(f"mssql+pyodbc:///?odbc_connect={params}", pool_size=2)
            return conexao_totvsdb
        # pylint: disable=broad-except
        except Exception as error:
            print(f"Error: {error}")
            return None

## Leitura do banco de dados


In [180]:
# database/db_read.py


# cSpell: disable=invalid-name
class Read(Connection):
    """
    Class Read
    Read data from the database and return a pandas dataframe
    Create query to be executed in the database
    """

    # pylint: disable=useless-super-delegation
    def __init__(self):
        """
        Constructor
        """
        super().__init__()

    def get_automacao_data(self, query: str) -> pd.DataFrame:
        """
        Get data from database AUTOMACAO and return a pandas dataframe.

        Parameters
        ----------
        query : str
            Query to be executed in the database

        Returns
        -------
        pandas dataframe
            Dataframe with the query result
        """
        connection = None
        try:
            connection = self.get_connection_automacao()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_automacao_query(self, table: str, where: str = None, orderby: str = None) -> str:
        """
        Create query to be executed in the database AUTOMACAO.

        Parameters
        ----------
        table : str
            Table name
        where : str
            Where clause (optional)
        orderby : str
            Order by clause (optional)

        Returns
        -------
        str
            Query to be executed in the database
        """
        query = f"SELECT * FROM AUTOMACAO.dbo.{table}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

    def get_totvsdb_data(self, query: str) -> pd.DataFrame:
        """
        Retrieves data from the TotvsDB database using the provided SQL query.

        Args:
            query (str): The SQL query to execute.

        Returns:
            pd.DataFrame: A pandas DataFrame containing the retrieved data.

        Raises:
            Exception: If an error occurs while retrieving the data.

        """
        connection = None
        try:
            connection = self.get_connection_totvsdb()
            data = pd.read_sql(query, connection)
            return data
        # pylint: disable=broad-except
        except Exception as e:
            print(f"Erro ao buscar dados: {e}")
            return None
        finally:
            if connection:
                connection.dispose()

    def create_totvsdb_query(
        self, select: str, table: str, join: str = None, where: str = None, orderby: str = None
    ) -> str:
        """
        Creates a SQL query string for querying the TOTVS database.

        Args:
            select (str): The SELECT clause of the query.
            table (str): The table name to query.
            join (str, optional): The JOIN clause of the query. Defaults to None.
            where (str, optional): The WHERE clause of the query. Defaults to None.
            orderby (str, optional): The ORDER BY clause of the query. Defaults to None.

        Returns:
            str: The SQL query string.
        """
        query = f"SELECT {select} FROM {table}"

        if join:
            query += f" {join}"

        if where:
            query += f" WHERE {where}"

        if orderby:
            query += f" ORDER BY {orderby}"

        return query

## Query para o banco de dados


In [181]:
# // database/get_data.py
# cSpell: disable=invalid-name
class GetData:
    """
    Essa classe é responsável por realizar a leitura dos dados do banco de dados.
    É utilizada para fazer a leitura em segundo plano, sem que o usuário perceba.
    """

    def __init__(self):
        self.db_read = Read()

    def get_data(self) -> tuple:
        """
        Realiza a leitura dos dados do banco de dados.
        Retorna na ordem: df_occ, df_info, df_cadastro
        """

        # Dia de hoje
        now = pd.to_datetime("today")

        # Encontrando primeiro dia do mês atual
        first_day = now.replace(day=1)

        # Mantendo apenas a data
        first_day = first_day.strftime("%Y-%m-%d")

        # NOTE: Se torna obsoleta com a nova maquina_ihm
        # Query para leitura dos dados de ocorrência
        query_occ = self.db_read.create_automacao_query(
            table="maquina_ocorrencia",
            where=f"data_registro <= '2024-04-30' and data_registro >= '2024-04-01'",  # f"data_registro >= '{first_day}'",
        )

        # Query para leitura dos dados de IHM
        query_ihm = self.db_read.create_automacao_query(
            table="maquina_ihm",
            where=f"data_registro >= '{first_day}'",
        )

        # Query para leitura dos dados de informações
        query_info = (
            "SELECT"
            " t1.maquina_id,"
            " (SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha,"
            " (SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2"
            " WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro"
            " ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica,"
            " t1.status,"
            " t1.turno,"
            " t1.contagem_total_ciclos,"
            " t1.contagem_total_produzido,"
            " t1.data_registro,"
            " t1.hora_registro"
            " FROM "
            " AUTOMACAO.dbo.maquina_info t1"
            f" WHERE data_registro <= '2024-04-30' and data_registro >= '2024-04-01'"  # f" WHERE data_registro >= '{first_day}'"
            " ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
        )

        # Query para leitura dos dados de de produção
        query_production = (
            "SELECT * "
            "FROM ( "
            "SELECT "
            "(SELECT TOP 1 t2.fabrica FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as fabrica, "
            "(SELECT TOP 1 t2.linha FROM AUTOMACAO.dbo.maquina_cadastro t2 "
            "WHERE t2.maquina_id = t1.maquina_id AND t2.data_registro <= t1.data_registro "
            "ORDER BY t2.data_registro DESC, t2.hora_registro DESC) as linha, "
            "t1.maquina_id, "
            "t1.turno, "
            "t1.status, "
            "t1.contagem_total_ciclos as total_ciclos, "
            "t1.contagem_total_produzido as total_produzido, "
            "t1.data_registro, "
            "t1.hora_registro, "
            "ROW_NUMBER() OVER ( "
            "PARTITION BY t1.data_registro, t1.turno, t1.maquina_id "
            "ORDER BY t1.data_registro DESC, t1.hora_registro DESC"
            ") AS rn "
            "FROM AUTOMACAO.dbo.maquina_info t1 "
            ") AS t "
            f" WHERE data_registro <= '2024-04-30' and data_registro >= '2024-04-01'"  # f" WHERE rn = 1 AND data_registro >= '{first_day}' AND hora_registro > '00:01'"
            " ORDER BY data_registro DESC, linha"
        )

        # Leitura dos dados
        df_occ = self.db_read.get_automacao_data(query_occ)
        df_ihm = self.db_read.get_automacao_data(query_ihm)
        df_info = self.db_read.get_automacao_data(query_info)
        df_info_production = self.db_read.get_automacao_data(query_production)

        # Verificando se os dados foram lidos corretamente
        if df_occ.empty or df_info.empty or df_info_production.empty:
            print("====== Erro na leitura dos dados do DB Automação ======")
            return None, None, None

        return df_occ, df_info, df_info_production


get_data = GetData()
df_occ, df_info, df_info_production = get_data.get_data()

## Testes de saída do banco de dados


In [182]:
df_occ.head(20)

,recno,maquina_id,motivo_id,problema,solucao,data_registro,hora_registro,usuario_id
0,5073,TMF005,03,,,2024-04-01,03:00:01,000441
1,5074,TMF011,03,,,2024-04-01,03:00:12,000441
2,5075,TMF015,03,,,2024-04-01,03:00:23,000441
3,5076,TMF012,03,,,2024-04-01,03:00:34,000441
4,5077,TMF003,06,,,2024-04-01,03:54:03,000441
5,5078,TMF001,06,,,2024-04-01,03:54:12,000441
6,5079,TMF006,06,,,2024-04-01,03:54:23,000441
7,5080,TMF003,03,,,2024-04-01,04:00:13,000441
8,5081,TMF001,03,,,2024-04-01,04:00:32,000441
9,5082,TMF006,03,,,2024-04-01,04:00:42,000441


## Testes de saída do banco de dados


In [183]:
df_info.head(20)

,maquina_id,linha,fabrica,status,turno,contagem_total_ciclos,contagem_total_produzido,data_registro,hora_registro
0,TMF004,9.0,1,false,VES,4.0,238.0,2024-04-30,23:59:14.830000
1,TMF014,8.0,1,false,VES,438.0,3212.0,2024-04-30,23:59:13.830000
2,TMF006,7.0,1,false,VES,370.0,318.0,2024-04-30,23:59:12.826666
3,TMF001,6.0,1,false,VES,482.0,336.0,2024-04-30,23:59:11.826666
4,TMF002,5.0,1,false,VES,300.0,234.0,2024-04-30,23:59:10.826666
5,TMF012,4.0,1,false,VES,48.0,38.0,2024-04-30,23:59:09.826666
6,TMF015,0.0,,false,VES,2.0,64.0,2024-04-30,23:59:08.826666
7,TMF011,2.0,1,false,VES,68.0,6.0,2024-04-30,23:59:07.826666
8,TMF003,1.0,1,false,VES,18.0,12.0,2024-04-30,23:59:06.823333
9,TMF004,9.0,1,false,VES,4.0,238.0,2024-04-30,23:57:14.826666


In [184]:
df_info_production.head(20)

,fabrica,linha,maquina_id,turno,status,total_ciclos,total_produzido,data_registro,hora_registro,rn
0,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:59:06.010000,1
1,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:57:06.003333,2
2,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:55:05.996666,3
3,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:53:05.990000,4
4,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:51:05.986666,5
5,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:49:05.980000,6
6,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:47:05.973333,7
7,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:45:05.966666,8
8,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:43:05.960000,9
9,,0.0,TMF015,NOT,false,156.0,64.0,2024-04-30,07:41:05.956666,10


# Limpeza de dados e análise exploratória


## Análise de dados - Clean Data


In [185]:
# service/clean_data.py


# cSpell: disable=invalid-name
class CleanData:
    """
    Classe para limpeza dos dados

    Métodos:
    --------
        clean_maq_info: Limpa os dados de status das máquinas
        clean_maq_occ: Limpa os dados de ocorrências das máquinas

    """

    def maq_info(self, info: pd.DataFrame) -> pd.DataFrame:
        """
        Processa as informações de uma máquina e retorna um DataFrame com os dados ajustados.

        Args:
            info (pd.DataFrame): DataFrame contendo as informações da máquina.

        Returns:
            pd.DataFrame: DataFrame com os dados ajustados da máquina.
        """

        # Ordenar dataframe
        df_info = info.sort_values(by=["maquina_id", "data_registro", "hora_registro", "turno"])

        # Criar coluna com data e hora unidos
        df_info["data_hora_registro"] = (
            df_info["data_registro"].astype(str)
            + " "
            + df_info["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Ajustar primeira entrada se for VES
        mask = (df_info["turno"] == "VES") & (
            df_info["maquina_id"] != df_info["maquina_id"].shift()
        )
        df_info["turno"] = np.where(mask, "NOT", df_info["turno"])

        # Ajustar data_hora para pd.datetime
        df_info["data_hora_registro"] = pd.to_datetime(df_info["data_hora_registro"])

        # Ajustar horário se turno for VES - ajusta para dia anterior e horário 23:59:59
        mask = (
            (df_info["turno"] == "VES")
            & (df_info["data_hora_registro"] != df_info["data_hora_registro"].shift())
            & (df_info["data_hora_registro"].dt.time > time(0, 0, 0))
            & (df_info["data_hora_registro"].dt.time < time(0, 5, 0))
        )
        df_info["data_hora_registro"] = np.where(
            mask,
            (df_info["data_hora_registro"] - pd.Timedelta(days=1)).dt.normalize()
            + pd.Timedelta(hours=23, minutes=59, seconds=59),
            df_info["data_hora_registro"],
        )

        # Criar nova coluna status_change para identificar mudança de status
        df_info["status_change"] = df_info["status"].ne(df_info["status"].shift())

        # Criar coluna para identificar a mudança de máquina
        df_info["maquina_change"] = df_info["maquina_id"].ne(df_info["maquina_id"].shift())

        # Criar coluna para identificar a mudança de turno
        df_info["turno_change"] = df_info["turno"].ne(df_info["turno"].shift())

        # Atualizar coluna change para incluir mudança de turno
        df_info["change"] = (
            df_info["status_change"] | df_info["maquina_change"] | df_info["turno_change"]
        )

        # Agrupar por maquina e identificar data e hora da última mudança de status
        df_info["change_time"] = (
            df_info.groupby("maquina_id")["data_hora_registro"].shift(0).where(df_info["change"])
        )

        # Feito para agrupar por maquina_id e turno e manter o ultimo registro de cada grupo
        df_info = (
            df_info.groupby(["maquina_id", "change_time"], observed=False)
            .agg(
                status=("status", "first"),
                turno=("turno", "first"),
                linha=("linha", "first"),
                fabrica=("fabrica", "first"),
                data_hora_registro=("data_hora_registro", "first"),
                contagem_total_ciclos=("contagem_total_ciclos", "last"),
                contagem_total_produzido=(
                    "contagem_total_produzido",
                    "last",
                ),
                change=("change", "first"),
                maquina_change=("maquina_change", "first"),
            )
            .reset_index()
        )

        # Criar nova coluna com a data_hora_final do status
        df_info["data_hora_final"] = (
            df_info.groupby("maquina_id", observed=False)["data_hora_registro"]
            .shift(-1)
            .where(~df_info["maquina_change"])
        )

        # Atualizar coluna data_hora_final onde maquina_change é True
        mask = df_info["maquina_change"]
        df_info["data_hora_final"] = np.where(
            mask, df_info["change_time"].shift(-1), df_info["data_hora_final"]
        )

        # Remover colunas desnecessárias
        df_info.drop(
            columns=[
                "maquina_change",
                "change",
                "change_time",
            ],
            inplace=True,
        )

        # Remover linhas onde data_hora_final é nulo
        df_info.dropna(subset=["data_hora_final"], inplace=True)

        # Cria nova coluna tempo_registro_min para calcular o tempo de registro em minutos
        df_info["tempo_registro_min"] = (
            pd.to_datetime(df_info["data_hora_final"])
            - pd.to_datetime(df_info["data_hora_registro"])
        ).dt.total_seconds() / 60

        # Arredondar tempo_registro_min e converter para inteiro
        df_info["tempo_registro_min"] = df_info["tempo_registro_min"].round(0).astype(int)

        # Ajustar tipos
        df_info = df_info.astype(
            {
                "maquina_id": "category",
                "status": "category",
                "turno": "category",
                "linha": "category",
                "fabrica": "category",
                "tempo_registro_min": int,
                "contagem_total_ciclos": int,
                "contagem_total_produzido": int,
            }
        )

        # Ajustar nomenclatura dos status
        df_info["status"] = np.where(
            (df_info["status"] == "true") & (df_info["tempo_registro_min"] < 10),
            "in_test",
            df_info["status"],
        )
        df_info["status"] = np.where(df_info["status"] == "true", "rodando", df_info["status"])
        df_info["status"] = np.where(df_info["status"] == "false", "parada", df_info["status"])

        # Ajustar tipo do status
        df_info["status"] = df_info["status"].astype("category")

        # Remover onde linha for 0
        df_info = df_info[df_info["linha"] != 0]

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def get_time_working(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de maquina rodando.
        """

        info = self.maq_info(data)

        df_info_rodando = info[info["status"] == "rodando"]

        # Agrupar por linha, turno e data e somar o tempo de registro
        df_info_rodando = (
            df_info_rodando.groupby(
                ["linha", "turno", "status", df_info_rodando["data_hora_registro"].dt.date],
                observed=False,
            )
            .agg(tempo_registro_min=("tempo_registro_min", "sum"))
            .reset_index()
        )

        # Remover linhas onde tempo_registro_min é menor que 0
        df_info_rodando = df_info_rodando[(df_info_rodando["tempo_registro_min"] > 0)]

        # Remover onde a linha for 0
        df_info_rodando = df_info_rodando[df_info_rodando["linha"] != 0]

        # Renomear colunas
        df_info_rodando.rename(
            columns={"status": "motivo_nome", "data_hora_registro": "data_registro"},
            inplace=True,
        )

        # Capitalizar o motivo nome
        df_info_rodando["motivo_nome"] = df_info_rodando["motivo_nome"].str.capitalize()

        return df_info_rodando

    def get_adjusted_stops_data(self, info: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de paradas ajustados de acordo com as regras definidas.

        Args:
            info (pd.DataFrame): O dataframe contendo os dados de paradas.

        Returns:
            pd.DataFrame: O dataframe com os dados de paradas ajustados.
        """
        # Certificar que data_hora_registro e data_hora_final são do tipo datetime
        info["data_hora_registro"] = pd.to_datetime(info["data_hora_registro"])
        info["data_hora_final"] = pd.to_datetime(info["data_hora_final"])

        # Ordenar por maquina_id e data_hora_registro
        df_info = info.sort_values(by=["maquina_id", "data_hora_registro"])

        # Criar coluna auxiliar para identificar a maquina rodando
        df_info["rodando"] = np.where(df_info["status"] == "rodando", 1, 0)

        # Unir grupos de paradas, levando em conta mudança de maquina e turno
        df_info["group"] = (
            (df_info["rodando"] != df_info["rodando"].shift())
            | (df_info["maquina_id"] != df_info["maquina_id"].shift())
            | (df_info["turno"] != df_info["turno"].shift())
            | (
                df_info["data_hora_registro"].dt.date
                != df_info["data_hora_registro"].shift().dt.date
            )
        ).cumsum()

        # Agregar por grupo
        df_info = (
            df_info.groupby(["group"], observed=False)
            .agg(
                maquina_id=("maquina_id", "first"),
                status=("status", "first"),
                turno=("turno", "first"),
                linha=("linha", "first"),
                fabrica=("fabrica", "first"),
                data_hora_registro=("data_hora_registro", "first"),
                data_hora_final=("data_hora_final", "last"),
                tempo_registro_min=("tempo_registro_min", "sum"),
                contagem_total_ciclos=("contagem_total_ciclos", "last"),
                contagem_total_produzido=("contagem_total_produzido", "last"),
            )
            .reset_index(drop=True)
        )

        # Alterar in_test para parada
        df_info["status"] = np.where(df_info["status"] == "in_test", "parada", df_info["status"])

        # Substituir valores nulos por np.nan
        df_info.fillna(value=np.nan, inplace=True)

        return df_info

    def dayofweek_adjust(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Incluir colunas para identificar sábados, domingos e feriados.

        Args:
            df (pd.DataFrame): DataFrame com os dados de paradas.

        Returns:
            pd.DataFrame: DataFrame com as colunas adicionadas.
        """

        # Garantir que data_hora_registro é do tipo datetime
        df["data_hora_registro"] = pd.to_datetime(df["data_hora_registro"])

        # Identificar sábados
        df["sabado"] = np.where(df["data_hora_registro"].dt.dayofweek == 5, 1, 0)

        # Identificar domingos
        df["domingo"] = np.where(df["data_hora_registro"].dt.dayofweek == 6, 1, 0)

        # Listar feriados
        # feriados = pd.read_csv(file_path)
        holidays = FERIADOS

        # Converter para datetime
        holidays["feriados"] = pd.to_datetime(holidays["feriados"])

        # Identificar feriados
        df["feriado"] = (
            df["data_hora_registro"]
            .dt.date.isin(pd.to_datetime(holidays["feriados"]).dt.date)
            .astype(int)
        )

        return df

    def get_maq_info_cleaned(self, df_info: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de paradas ajustados de acordo com as regras definidas.

        Args:
            df_info (pd.DataFrame): O dataframe contendo os dados de paradas.

        Returns:
            pd.DataFrame: O dataframe com os dados de paradas ajustados.
        """

        # Ajustar dados de maquina_info
        df_info = self.maq_info(df_info)

        # Ajustar dados de paradas
        df_info = self.get_adjusted_stops_data(df_info)

        # Incluir colunas para identificar sábados, domingos e feriados
        df_info = self.dayofweek_adjust(df_info)

        return df_info

    def get_maq_occ_cleaned(self, df_occ: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de ocorrências ajustados de acordo com as regras definidas.

        Args:
            df_occ (pd.DataFrame): O dataframe contendo os dados de ocorrências.

        Returns:
            pd.DataFrame: O dataframe com os dados de ocorrências ajustados.
        """

        # Motivos de Parada
        motivos = {
            1: "Ajustes",
            2: "Troca de Bobina",
            3: "Refeição",
            4: "Reunião",
            5: "Café e Ginástica Laboral",
            6: "Limpeza",
            7: "Manutenção Elétrica",
            8: "Manutenção Mecânica",
            9: "Material em Falta",
            10: "Setup de Sabor",
            11: "Setup de Tamanho",
            12: "Parada Programada",
            13: "Intervenção de Qualidade",
            14: "Linha Cheia",
            15: "Treinamento",
            16: "Limpeza Industrial",
            17: "Troca de Filme",
        }

        # Ajustar coluna motivo_id para int
        df_occ = df_occ.astype({"motivo_id": int})

        # Unir data_registro e hora_registro
        df_occ["data_hora_registro"] = (
            df_occ["data_registro"].astype(str)
            + " "
            + df_occ["hora_registro"].astype(str).str.split(".").str[0]
        )

        # Ajustar data_hora_registro para datetime
        df_occ["data_hora_registro"] = pd.to_datetime(df_occ["data_hora_registro"])

        # Criar coluna com motivo_nome com base no dicionário motivos
        df_occ["motivo_nome"] = df_occ["motivo_id"].map(motivos)

        # Ajustar "problema" e "solucao" se a string estiver vazia
        df_occ["problema"] = np.where(df_occ["problema"] == "", np.nan, df_occ["problema"])
        df_occ["solucao"] = np.where(df_occ["solucao"] == "", np.nan, df_occ["solucao"])

        # Copiar motivo_nome para problema caso problema seja nulo e motivo_id não seja 1,7,8,9,14
        df_occ["problema"] = np.where(
            (df_occ["problema"].isnull())
            & (
                ~df_occ["motivo_id"].isin(
                    [
                        1,
                        7,
                        8,
                        9,
                        14,
                    ]
                )
            ),
            df_occ["motivo_nome"],
            df_occ["problema"],
        )

        # Ajustar ordem das colunas e seus tipos
        df_occ = df_occ.reindex(
            columns=[
                "maquina_id",
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro",
                "usuario_id",
            ]
        )
        df_occ = df_occ.astype(
            {
                "maquina_id": "category",
                "motivo_id": int,
                "motivo_nome": "category",
                "problema": str,
                "solucao": "category",
                "data_hora_registro": "datetime64[ns]",
                "usuario_id": "category",
            }
        )

        return df_occ

    def get_maq_production_cleaned(self, df_production: pd.DataFrame) -> pd.DataFrame:
        """
        Retorna os dados de produção ajustados de acordo com as regras definidas.

        Args:
            df_production (pd.DataFrame): O dataframe contendo os dados de produção.

        Returns:
            pd.DataFrame: O dataframe com os dados de produção ajustados.
        """

        # Incluir coluna turno_number para ordenar os turnos
        df_production["turno_number"] = df_production["turno"].map({"MAT": 2, "VES": 3, "NOT": 1})

        # Ordenar por maquina_id, data_registro e turno_number
        df_production.sort_values(by=["maquina_id", "data_registro", "turno_number"], inplace=True)

        # Remover coluna turno_number
        df_production.drop(columns=["turno_number"], inplace=True)

        # Ajustar tipos
        df_production = df_production.astype(
            {
                "maquina_id": "category",
                "linha": "category",
                "turno": "category",
                "total_ciclos": int,
                "total_produzido": int,
                "data_registro": "datetime64[ns]",
            }
        )

        # Ajustar o index
        df_production.reset_index(drop=True, inplace=True)

        return df_production


clean_data = CleanData()

## Análise de dados - Retorno de Dados


### Máquina Info


In [186]:
df_info_clean = clean_data.maq_info(df_info.copy())
df_info_clean.head(28)

,maquina_id,status,turno,linha,fabrica,data_hora_registro,contagem_total_ciclos,contagem_total_produzido,data_hora_final,tempo_registro_min
0,,parada,NOT,NaN,NaN,2024-04-19 04:00:21,0,0,2024-04-19 16:00:14,720
1,,parada,VES,NaN,NaN,2024-04-19 16:00:14,0,0,2024-04-23 17:19:42,5839
2,,in_test,VES,NaN,NaN,2024-04-23 17:19:42,48,48,2024-04-23 17:19:45,0
3,,parada,VES,NaN,NaN,2024-04-23 17:19:45,0,0,2024-04-23 17:19:46,0
4,,in_test,VES,NaN,NaN,2024-04-23 17:19:46,42,40,2024-04-23 17:19:47,0
5,,parada,VES,NaN,NaN,2024-04-23 17:19:47,0,0,2024-04-23 17:19:48,0
6,,in_test,VES,NaN,NaN,2024-04-23 17:19:48,38,0,2024-04-23 17:19:50,0
7,,parada,VES,NaN,NaN,2024-04-23 17:19:50,0,0,2024-04-23 17:21:42,2
8,,in_test,VES,NaN,NaN,2024-04-23 17:21:42,96,96,2024-04-23 17:21:45,0
9,,parada,VES,NaN,NaN,2024-04-23 17:21:45,0,0,2024-04-23 17:21:46,0


In [187]:
df_info_stops = clean_data.get_adjusted_stops_data(df_info_clean.copy())
df_info_stops.head(28)

,maquina_id,status,turno,linha,fabrica,data_hora_registro,data_hora_final,tempo_registro_min,contagem_total_ciclos,contagem_total_produzido
0,,parada,NOT,NaN,NaN,2024-04-19 04:00:21,2024-04-19 16:00:14,720,0,0
1,,parada,VES,NaN,NaN,2024-04-19 16:00:14,2024-04-23 17:19:42,5839,0,0
2,,parada,VES,NaN,NaN,2024-04-23 17:19:42,2024-04-23 18:00:02,40,0,0
3,TMF001,parada,NOT,6.0,1,2024-04-01 00:00:23,2024-04-01 00:12:23,12,100,0
4,TMF001,rodando,NOT,6.0,1,2024-04-01 00:12:23,2024-04-01 00:24:23,12,42,14
5,TMF001,parada,NOT,6.0,1,2024-04-01 00:24:23,2024-04-01 00:34:23,10,316,268
6,TMF001,rodando,NOT,6.0,1,2024-04-01 00:34:23,2024-04-01 01:36:23,62,358,310
7,TMF001,parada,NOT,6.0,1,2024-04-01 01:36:23,2024-04-01 01:40:23,4,1680,1626
8,TMF001,rodando,NOT,6.0,1,2024-04-01 01:40:23,2024-04-01 03:50:24,130,1716,1662
9,TMF001,parada,NOT,6.0,1,2024-04-01 03:50:24,2024-04-01 05:04:24,74,4656,4574


In [188]:
df_info_cleaned = clean_data.get_maq_info_cleaned(df_info.copy())
df_info_cleaned.head(28)

,maquina_id,status,turno,linha,fabrica,data_hora_registro,data_hora_final,tempo_registro_min,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado
0,,parada,NOT,NaN,NaN,2024-04-19 04:00:21,2024-04-19 16:00:14,720,0,0,0,0,0
1,,parada,VES,NaN,NaN,2024-04-19 16:00:14,2024-04-23 17:19:42,5839,0,0,0,0,0
2,,parada,VES,NaN,NaN,2024-04-23 17:19:42,2024-04-23 18:00:02,40,0,0,0,0,0
3,TMF001,parada,NOT,6.0,1,2024-04-01 00:00:23,2024-04-01 00:12:23,12,100,0,0,0,0
4,TMF001,rodando,NOT,6.0,1,2024-04-01 00:12:23,2024-04-01 00:24:23,12,42,14,0,0,0
5,TMF001,parada,NOT,6.0,1,2024-04-01 00:24:23,2024-04-01 00:34:23,10,316,268,0,0,0
6,TMF001,rodando,NOT,6.0,1,2024-04-01 00:34:23,2024-04-01 01:36:23,62,358,310,0,0,0
7,TMF001,parada,NOT,6.0,1,2024-04-01 01:36:23,2024-04-01 01:40:23,4,1680,1626,0,0,0
8,TMF001,rodando,NOT,6.0,1,2024-04-01 01:40:23,2024-04-01 03:50:24,130,1716,1662,0,0,0
9,TMF001,parada,NOT,6.0,1,2024-04-01 03:50:24,2024-04-01 05:04:24,74,4656,4574,0,0,0


### Ocorrências


In [189]:
df_maq_occ_cleaned = clean_data.get_maq_occ_cleaned(df_occ.copy())
df_maq_occ_cleaned.head(28)

,maquina_id,motivo_id,motivo_nome,problema,solucao,data_hora_registro,usuario_id
0,TMF005,3,Refeição,Refeição,NaN,2024-04-01 03:00:01,000441
1,TMF011,3,Refeição,Refeição,NaN,2024-04-01 03:00:12,000441
2,TMF015,3,Refeição,Refeição,NaN,2024-04-01 03:00:23,000441
3,TMF012,3,Refeição,Refeição,NaN,2024-04-01 03:00:34,000441
4,TMF003,6,Limpeza,Limpeza,NaN,2024-04-01 03:54:03,000441
5,TMF001,6,Limpeza,Limpeza,NaN,2024-04-01 03:54:12,000441
6,TMF006,6,Limpeza,Limpeza,NaN,2024-04-01 03:54:23,000441
7,TMF003,3,Refeição,Refeição,NaN,2024-04-01 04:00:13,000441
8,TMF001,3,Refeição,Refeição,NaN,2024-04-01 04:00:32,000441
9,TMF006,3,Refeição,Refeição,NaN,2024-04-01 04:00:42,000441


### Produção


In [190]:
df_maq_info_production_cleaned = clean_data.get_maq_production_cleaned(df_info_production.copy())
df_maq_info_production_cleaned.head(28)

,fabrica,linha,maquina_id,turno,status,total_ciclos,total_produzido,data_registro,hora_registro,rn
0,None,NaN,,NOT,false,0,0,2024-04-19,04:00:21.270000,1
1,None,NaN,,VES,false,0,0,2024-04-19,16:45:46.960000,1
2,None,NaN,,VES,false,0,0,2024-04-19,16:45:45.960000,2
3,None,NaN,,VES,false,0,0,2024-04-19,16:45:44.960000,3
4,None,NaN,,VES,false,0,0,2024-04-19,16:45:42.960000,4
5,None,NaN,,VES,false,0,0,2024-04-19,16:45:41.956666,5
6,None,NaN,,VES,false,0,0,2024-04-19,16:45:40.956666,6
7,None,NaN,,VES,false,0,0,2024-04-19,16:45:39.956666,7
8,None,NaN,,VES,false,0,0,2024-04-19,16:45:38.956666,8
9,None,NaN,,VES,false,0,0,2024-04-19,16:43:46.956666,9


### Tempo Rodando


In [191]:
df_time_working = clean_data.get_time_working(df_info.copy())
df_time_working.head(28)

,linha,turno,motivo_nome,data_registro,tempo_registro_min
297,1.0,MAT,Rodando,2024-04-01,266
298,1.0,MAT,Rodando,2024-04-02,422
299,1.0,MAT,Rodando,2024-04-03,280
300,1.0,MAT,Rodando,2024-04-04,220
301,1.0,MAT,Rodando,2024-04-05,216
302,1.0,MAT,Rodando,2024-04-06,272
304,1.0,MAT,Rodando,2024-04-08,314
305,1.0,MAT,Rodando,2024-04-09,230
306,1.0,MAT,Rodando,2024-04-10,290
307,1.0,MAT,Rodando,2024-04-11,260


# Unindo informações de máquina e ocorrências


## Unindo


In [192]:
# service/join_data.py
# cSpell: disable=invalid-name


class JoinData:
    """
    Classe para unir os dados de cadastro, info e ocorrência
    """

    # Função Auxiliar
    def move_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Move as colunas para a posição desejada.
        """

        # Adiciona uma verificação para garantir que não haja valores nulos
        mask = (
            df[["data_hora_registro_occ", "data_hora_registro", "data_hora_final"]]
            .notna()
            .all(axis=1)
        )

        df["data_hora_registro_occ"] = df["data_hora_registro_occ"].replace(np.nan, pd.NaT)
        df = df.astype({"data_hora_registro_occ": "datetime64[ns]"})

        # Calcula a diferença de data_hora_registro_occ e data_hora_registro e data_hora_final
        df["diff_registro"] = np.where(
            mask,
            (df["data_hora_registro_occ"] - df["data_hora_registro"]).dt.total_seconds().abs(),
            np.nan,
        )
        df["diff_final"] = np.where(
            mask,
            (df["data_hora_registro_occ"] - df["data_hora_final"]).dt.total_seconds().abs(),
            np.nan,
        )

        # Encontra qual das duas diferenças é menor
        mask_idxmin = df[["diff_registro", "diff_final"]].notna().any(axis=1)
        df.loc[mask & mask_idxmin, "closest"] = df.loc[
            mask & mask_idxmin, ["diff_registro", "diff_final"]
        ].idxmin(
            axis=1
        )  # removendo erro:
        # FutureWarning: The behavior of DataFrame.idxmin with all-NA values,
        # or any-NA and skipna=False, is deprecated.
        # In a future version this will raise ValueError

        columns = [
            "motivo_id",
            "motivo_nome",
            "problema",
            "solucao",
            "usuario_id_occ",
        ]

        # Move a ocorrência para a linha anterior se a data_hora_registro for mais próxima
        mask = (
            (df["closest"].shift(-1) == "diff_registro")
            & (df["status"].shift(-1) == "rodando")
            & df["motivo_id"].isnull()
        )
        for column in columns:
            df[column] = np.where(mask, df[column].shift(-1), df[column])
            df[column] = np.where(mask.shift(1), np.nan, df[column])

        # Tratamento separado para a coluna "data_hora_registro_occ"
        df["data_hora_registro_occ"] = np.where(
            mask, df["data_hora_registro_occ"].shift(-1), df["data_hora_registro_occ"]
        )
        df["data_hora_registro_occ"] = np.where(mask.shift(1), pd.NaT, df["data_hora_registro_occ"])

        # Move a ocorrência para a linha seguinte se a data_hora_final for mais próxima
        mask = (
            (df["closest"].shift(1) == "diff_final")
            & (df["status"].shift(1) == "rodando")
            & df["motivo_id"].isnull()
        )
        for column in columns:
            df[column] = np.where(mask, df[column].shift(1), df[column])
            df[column] = np.where(mask.shift(-1), np.nan, df[column])

        # Tratamento separado para a coluna "data_hora_registro_occ"
        df["data_hora_registro_occ"] = np.where(
            mask, df["data_hora_registro_occ"].shift(1), df["data_hora_registro_occ"]
        )
        df["data_hora_registro_occ"] = np.where(
            mask.shift(-1), pd.NaT, df["data_hora_registro_occ"]
        )

        return df

    def join_info_occ(self, df_occ: pd.DataFrame, df_info: pd.DataFrame) -> pd.DataFrame:
        """
        Junta os dados de ocorrências e paradas.

        Args:
            df_occ (pd.DataFrame): DataFrame com os dados de ocorrências.
            df_info (pd.DataFrame): DataFrame com os dados de paradas.

        Returns:
            pd.DataFrame: DataFrame com os dados de ocorrências e paradas juntos.
        """

        # Garantir que as colunas com datas sejam datetime
        df_occ["data_hora_registro"] = pd.to_datetime(df_occ["data_hora_registro"])
        df_info["data_hora_registro"] = pd.to_datetime(df_info["data_hora_registro"])
        df_info["data_hora_final"] = pd.to_datetime(df_info["data_hora_final"])

        # Juntar os dados de ocorrências e paradas
        def merge_rows(row):
            """
            Função para juntar os dados de ocorrências e paradas.
            """
            mask = (
                (df_occ["maquina_id"] == row["maquina_id"])
                & (df_occ["data_hora_registro"] >= row["data_hora_registro"])
                & (df_occ["data_hora_registro"] <= row["data_hora_final"])
            )  # mask para identificar as paradas que ocorreram durante a ocorrência

            # Criar dataframe com as paradas que ocorreram durante a ocorrência
            if df_occ[mask].shape[0] > 0:
                return pd.Series(
                    [
                        df_occ[mask]["motivo_id"].values[0],
                        df_occ[mask]["motivo_nome"].values[0],
                        df_occ[mask]["problema"].values[0],
                        df_occ[mask]["solucao"].values[0],
                        df_occ[mask]["data_hora_registro"].values[0],
                        df_occ[mask]["usuario_id"].values[0],
                    ],
                )

            return pd.Series([None, None, None, None, None, None])

        # Aplicar a função merge_rows
        df_info[
            [
                "motivo_id",
                "motivo_nome",
                "problema",
                "solucao",
                "data_hora_registro_occ",
                "usuario_id_occ",
            ]
        ] = df_info.apply(merge_rows, axis=1)

        # Reordenar colunas
        df_info = df_info.reindex(
            columns=[
                "maquina_id",
                "linha",
                "fabrica",
                "turno",
                "data_hora_registro",
                "tempo_registro_min",
                "data_hora_final",
                "status",
                "motivo_id",
                "data_hora_registro_occ",
                "motivo_nome",
                "problema",
                "solucao",
                "usuario_id_occ",
                "contagem_total_ciclos",
                "contagem_total_produzido",
                "sabado",
                "domingo",
                "feriado",
            ]
        )

        # Ordenar por linha, data_hora_registro
        df_info.sort_values(by=["linha", "data_hora_registro"], inplace=True)

        # Ajustar problema e solução caso seja "nan"
        df_info["problema"] = df_info["problema"].astype(str)
        df_info["solucao"] = df_info["solucao"].astype(str)
        df_info["problema"] = np.where(df_info["problema"] == "nan", np.nan, df_info["problema"])
        df_info["solucao"] = np.where(df_info["solucao"] == "nan", np.nan, df_info["solucao"])

        # Remove a linha com tempo_registro_min negativo ou 0
        df_info = df_info[df_info["tempo_registro_min"] > 0]

        df_info = self.move_columns(df_info)

        # Remove colunas desnecessárias
        df_info.drop(
            columns=[
                "diff_registro",
                "diff_final",
                "closest",
            ],
            inplace=True,
        )

        # Ajustar motivo id
        df_info["motivo_id"] = df_info["motivo_id"].fillna(np.nan)

        # Ajustar em problema, solucao, usuario_id_occ e motivo_nome
        df_info["problema"] = df_info["problema"].fillna("")
        df_info["solucao"] = df_info["solucao"].fillna("")
        df_info["usuario_id_occ"] = np.where(
            df_info["usuario_id_occ"] == pd.NaT, np.nan, df_info["usuario_id_occ"]
        )
        df_info["motivo_nome"] = np.where(
            df_info["motivo_nome"] == pd.NaT, np.nan, df_info["motivo_nome"]
        )
        # Mudar de np.nan para pd.NaT em data_hora_registro_occ
        df_info["data_hora_registro_occ"] = np.where(
            df_info["data_hora_registro_occ"].isnull(), pd.NaT, df_info["data_hora_registro_occ"]
        )

        # Corrigir os formatos das colunas
        df_info = df_info.astype(
            {
                "data_hora_registro_occ": "datetime64[ns]",
                "motivo_nome": "category",
                "problema": str,
                "solucao": str,
                "usuario_id_occ": "category",
            }
        )

        # Ajustar problema para Domingo, Sábado e Feriado se motivo_id for 12
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["domingo"] == 1),
            "Domingo",
            df_info["problema"],
        )
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["sabado"] == 1),
            "Sábado",
            df_info["problema"],
        )
        df_info["problema"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["feriado"] == 1),
            "Feriado",
            df_info["problema"],
        )

        # ------------------------------ Ajustes Marcação de Paradas ---------------------------- #

        # Ajustar motivo 12 para Sábado, Domingo e Feriado
        condition = (
            (df_info["motivo_id"].isnull())
            & (df_info["status"] == "parada")
            & (df_info["tempo_registro_min"] > 400)
        )
        condition_sabado = condition & (df_info["sabado"] == 1)
        df_info["motivo_id"] = np.where(condition_sabado, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_sabado, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_sabado, "Sábado", df_info["problema"])

        condition_domingo = condition & (df_info["domingo"] == 1)
        df_info["motivo_id"] = np.where(condition_domingo, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_domingo, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_domingo, "Domingo", df_info["problema"])

        condition_feriado = condition & (df_info["feriado"] == 1)
        df_info["motivo_id"] = np.where(condition_feriado, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_feriado, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_feriado, "Feriado", df_info["problema"])

        # Se o tempo de registro for maior que 480 mudar para 480
        df_info["tempo_registro_min"] = np.where(
            df_info["tempo_registro_min"] > 480, 480, df_info["tempo_registro_min"]
        )

        # Ajusta para parada programada qdo não tem o motivo e fica parada todo turno
        mask = (df_info["motivo_id"].isnull()) & (df_info["tempo_registro_min"] >= 478)
        df_info["motivo_id"] = np.where(mask, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(mask, "Parada Programada", df_info["motivo_nome"])

        # ---- Busca a última parada caso não tenha motivo e seja a primeira parada do turno ---- #
        # Ordena o DataFrame por 'maquina_id' e 'turno'
        df_info.sort_values(by=["maquina_id", "data_hora_registro"], inplace=True)

        # Cria colunas temporárias com os valores do último turno
        df_info["motivo_id_last"] = df_info.groupby("maquina_id", observed=False)[
            "motivo_id"
        ].shift()
        df_info["motivo_nome_last"] = df_info.groupby("maquina_id", observed=False)[
            "motivo_nome"
        ].shift()
        df_info["problema_last"] = df_info.groupby("maquina_id", observed=False)["problema"].shift()
        df_info["solucao_last"] = df_info.groupby("maquina_id", observed=False)["solucao"].shift()

        # Cria a máscara para as linhas que atendem às condições
        mask = (df_info["tempo_registro_min"] > 200) & df_info["motivo_id"].isnull()

        # Aplica a máscara e substitui os valores nas colunas originais
        df_info.loc[mask, "motivo_id"] = df_info.loc[mask, "motivo_id_last"]
        df_info.loc[mask, "motivo_nome"] = df_info.loc[mask, "motivo_nome_last"]
        df_info.loc[mask, "problema"] = df_info.loc[mask, "problema_last"]
        df_info.loc[mask, "solucao"] = df_info.loc[mask, "solucao_last"]

        # Remove as colunas temporárias
        df_info = df_info.drop(
            columns=["motivo_id_last", "motivo_nome_last", "problema_last", "solucao_last"]
        )
        # -------------------------------------------------------------------------------- #

        # Se for motivo 12 e parada for 478 minutos ou mais, ajustar para 480
        df_info["tempo_registro_min"] = np.where(
            (df_info["motivo_id"] == 12) & (df_info["tempo_registro_min"] >= 478),
            480,
            df_info["tempo_registro_min"],
        )

        # Se o motivo for 6, o tempo de registro for maior que 200 e for sábado,
        # alterar o motivo para 16 e o motivo_nome para "Limpeza Industrial"
        m6_mask = (
            (df_info["motivo_id"] == 6)
            & (df_info["tempo_registro_min"] > 200)
            & (df_info["sabado"] == 1)
        )
        df_info["motivo_id"] = np.where(m6_mask, 16, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            (df_info["motivo_id"] == 16) & (df_info["sabado"] == 1),
            "Limpeza Industrial",
            df_info["motivo_nome"],
        )

        # Se for domingo e o tempo de registro for 480, ajustar para
        # motivo_id 12, motivo_nome "Parada Programada" e problema "Domingo"
        condition_mask = (df_info["domingo"] == 1) & (df_info["tempo_registro_min"] == 480)
        df_info["motivo_id"] = np.where(condition_mask, 12, df_info["motivo_id"])
        df_info["motivo_nome"] = np.where(
            condition_mask, "Parada Programada", df_info["motivo_nome"]
        )
        df_info["problema"] = np.where(condition_mask, "Domingo", df_info["problema"])

        # Remover as linhas onde a linha é 0
        df_info = df_info[df_info["linha"] != 0]

        # Remover as linhas onde status é rodando
        df_info = df_info[df_info["status"] != "rodando"]

        mask_no_energy = (
            (df_info["status"] == "parada")
            & (df_info["tempo_registro_min"] < 480)
            & (df_info["status"].shift(-1) == "parada")
            & (df_info["tempo_registro_min"].shift(-1) == 480)
            & (df_info["status"].shift() == "parada")
            & (df_info["tempo_registro_min"].shift() == 480)
        )
        df_info.loc[mask_no_energy, "tempo_registro_min"] = 480

        # Ajustar o index
        df_info.reset_index(drop=True, inplace=True)

        return df_info

    def problems_adjust(self, df: pd.DataFrame, threshold=88) -> pd.DataFrame:
        """
        Ajusta os problemas no DataFrame fornecido,
        mapeando problemas semelhantes para um nome comum.

        Args:
            df (pd.DataFrame): O DataFrame contendo os problemas a serem ajustados.
            threshold (int, opcional): O limite de similaridade para combinar problemas.
            Por padrão é 88.

        Returns:
            pd.DataFrame: O DataFrame com problemas ajustados.
        """
        # Encontrar problemas únicos
        unique_problems = df["problema"].unique()
        problem_mapping = {}

        # Criar um dicionário para mapear os problemas
        for problem in unique_problems:
            if problem and problem not in problem_mapping:
                problem = str(problem).capitalize()

                # Corrigir erros básicos de digitação
                problem = problem.replace("Beckup", "Backup")
                problem = problem.replace("Becukp", "Backup")
                problem = problem.replace("Stm", "Atm")

                matches = process.extract(problem, unique_problems, limit=len(unique_problems))

                # Encontrar os problemas com maior similaridade
                similar_problems = [match[0] for match in matches if match[1] >= threshold]

                # Criar um dicionário com os problemas similares
                for similar_problem in similar_problems:
                    problem_mapping[similar_problem] = problem

        # Mapear os problemas
        df["problema"] = df["problema"].map(problem_mapping)

        return df


join_data = JoinData()

## Saída de dados


In [193]:
df_info_occ = join_data.join_info_occ(df_maq_occ_cleaned.copy(), df_info_cleaned.copy())
df_info_occ

,maquina_id,linha,fabrica,turno,data_hora_registro,tempo_registro_min,data_hora_final,status,motivo_id,data_hora_registro_occ,motivo_nome,problema,solucao,usuario_id_occ,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado
0,,NaN,NaN,NOT,2024-04-19 04:00:21,480,2024-04-19 16:00:14,parada,12.0,NaT,Parada Programada,None,None,NaN,0,0,0,0,0
1,,NaN,NaN,VES,2024-04-19 16:00:14,480,2024-04-23 17:19:42,parada,12.0,NaT,Parada Programada,None,None,NaN,0,0,0,0,0
2,,NaN,NaN,VES,2024-04-23 17:19:42,40,2024-04-23 18:00:02,parada,NaN,NaT,NaN,,,NaN,0,0,0,0,0
3,TMF001,6.0,1,NOT,2024-04-01 00:00:23,12,2024-04-01 00:12:23,parada,NaN,NaT,NaN,None,None,NaN,100,0,0,0,0
4,TMF001,6.0,1,NOT,2024-04-01 00:24:23,10,2024-04-01 00:34:23,parada,NaN,NaT,NaN,None,None,NaN,316,268,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3856,TMF015,3.0,1,MAT,2024-04-29 14:39:02,4,2024-04-29 14:43:02,parada,NaN,NaT,NaN,None,None,NaN,2910,2880,0,0,0
3857,TMF015,3.0,1,MAT,2024-04-29 14:59:02,4,2024-04-29 15:03:02,parada,NaN,NaT,NaN,None,None,NaN,3150,3118,0,0,0
3858,TMF015,3.0,1,MAT,2024-04-29 15:31:03,6,2024-04-29 15:37:03,parada,NaN,NaT,NaN,None,None,NaN,3732,3692,0,0,0
3859,TMF015,3.0,1,MAT,2024-04-29 15:53:03,8,2024-04-29 16:01:03,parada,NaN,NaT,NaN,None,None,NaN,4104,4058,0,0,0


In [194]:
df_maq_info_occ_combined = join_data.problems_adjust(df_info_occ.copy())
df_maq_info_occ_combined.head(28)

,maquina_id,linha,fabrica,turno,data_hora_registro,tempo_registro_min,data_hora_final,status,motivo_id,data_hora_registro_occ,motivo_nome,problema,solucao,usuario_id_occ,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado
0,,NaN,NaN,NOT,2024-04-19 04:00:21,480,2024-04-19 16:00:14,parada,12.0,NaT,Parada Programada,None,None,NaN,0,0,0,0,0
1,,NaN,NaN,VES,2024-04-19 16:00:14,480,2024-04-23 17:19:42,parada,12.0,NaT,Parada Programada,None,None,NaN,0,0,0,0,0
2,,NaN,NaN,VES,2024-04-23 17:19:42,40,2024-04-23 18:00:02,parada,NaN,NaT,NaN,NaN,,NaN,0,0,0,0,0
3,TMF001,6.0,1,NOT,2024-04-01 00:00:23,12,2024-04-01 00:12:23,parada,NaN,NaT,NaN,None,None,NaN,100,0,0,0,0
4,TMF001,6.0,1,NOT,2024-04-01 00:24:23,10,2024-04-01 00:34:23,parada,NaN,NaT,NaN,None,None,NaN,316,268,0,0,0
5,TMF001,6.0,1,NOT,2024-04-01 01:36:23,4,2024-04-01 01:40:23,parada,NaN,NaT,NaN,None,None,NaN,1680,1626,0,0,0
6,TMF001,6.0,1,NOT,2024-04-01 03:50:24,74,2024-04-01 05:04:24,parada,6.0,2024-04-01 03:54:12,Limpeza,Limpeza,,000441,4656,4574,0,0,0
7,TMF001,6.0,1,NOT,2024-04-01 06:38:24,2,2024-04-01 06:40:24,parada,NaN,NaT,NaN,None,None,NaN,6674,6582,0,0,0
8,TMF001,6.0,1,NOT,2024-04-01 07:26:24,2,2024-04-01 07:28:24,parada,NaN,NaT,NaN,None,None,NaN,7726,7628,0,0,0
9,TMF001,6.0,1,NOT,2024-04-01 07:42:24,10,2024-04-01 07:52:25,parada,NaN,NaT,NaN,None,None,NaN,8230,8124,0,0,0


# Ajuste para salvar o histórico no banco de dados sqlite


In [195]:
df_maq_info_cadastro = df_maq_info_occ_combined.copy()

# Remover colunas desnecessárias
df_maq_info_cadastro.drop(
    columns=[
        "motivo_id",
        "solucao",
        "contagem_total_ciclos",
        "contagem_total_produzido",
        "sabado",
        "domingo",
        "feriado",
    ],
    inplace=True,
)

# Renomear Colunas
df_maq_info_cadastro.rename(
    columns={
        "data_hora_registro": "data_hora",
        "tempo_registro_min": "tempo",
        "motivo_nome": "motivo",
        "usuario_id_occ": "operador_id",
    },
    inplace=True,
)

# Criar coluna data_registro à partir da data_hora
df_maq_info_cadastro["data_registro"] = df_maq_info_cadastro["data_hora"].dt.date

# Criar coluna hora_registro à partir da data_hora
df_maq_info_cadastro["hora_registro"] = df_maq_info_cadastro["data_hora"].dt.time

# Criar colunas com valores Nulos
df_maq_info_cadastro["equipamento"] = None
df_maq_info_cadastro["causa"] = None
df_maq_info_cadastro["os_numero"] = None
df_maq_info_cadastro["data_registro_ihm"] = df_maq_info_cadastro["data_registro"]
df_maq_info_cadastro["hora_registro_ihm"] = df_maq_info_cadastro["hora_registro"]

# Reordenar as colunas
df_maq_info_cadastro = df_maq_info_cadastro[
    [
        "fabrica",
        "linha",
        "maquina_id",
        "turno",
        "status",
        "data_registro",
        "hora_registro",
        "motivo",
        "equipamento",
        "problema",
        "causa",
        "os_numero",
        "operador_id",
        "data_registro_ihm",
        "hora_registro_ihm",
        "data_hora",
        "data_hora_final",
        "tempo",
    ]
]

# Ajustar tipos
df_maq_info_cadastro = df_maq_info_cadastro.astype(
    {
        "fabrica": "category",
        "linha": int,
        "maquina_id": "category",
        "turno": "category",
        "status": "category",
        "data_registro": "datetime64[ns]",
        "hora_registro": "object",
        "motivo": "category",
        "equipamento": "category",
        "problema": str,
        "causa": "category",
        "os_numero": "category",
        "operador_id": "category",
        "data_registro_ihm": "datetime64[ns]",
        "hora_registro_ihm": "object",
        "data_hora": "datetime64[ns]",
        "data_hora_final": "datetime64[ns]",
        "tempo": int,
    }
)

df_maq_info_cadastro.dtypes

ValueError: Cannot convert float NaN to integer: Error while type casting for column 'linha'

In [ ]:
df_working_minutes = df_time_working.copy()

# Adicionar coluna
df_working_minutes["maquina_id"] = None

# Reordenar as colunas
df_working_minutes = df_working_minutes[
    [
        "maquina_id",
        "linha",
        "turno",
        "data_registro",
        "motivo_nome",
        "tempo_registro_min",
    ]
]

# Renomear colunas
df_working_minutes.rename(
    columns={
        "motivo_nome": "motivo",
        "tempo_registro_min": "tempo",
    },
    inplace=True,
)

# Ajustar tipos
df_working_minutes = df_working_minutes.astype(
    {
        "linha": int,
        "data_registro": "datetime64[ns]",
    }
)

df_working_minutes.dtypes

maquina_id               object
linha                     int32
turno                  category
data_registro    datetime64[ns]
motivo                   object
tempo                     int32
dtype: object

In [ ]:
df_maq_info_prod_cad_cleaned = df_maq_info_production_cleaned.copy()

# Remover colunas desnecessárias
df_maq_info_prod_cad_cleaned.drop(
    columns=[
        "status",
        "rn",
    ],
    inplace=True,
)

# Ajustar tipos
df_maq_info_prod_cad_cleaned = df_maq_info_prod_cad_cleaned.astype(
    {
        "linha": int,
        "total_ciclos": float,
        "total_produzido": float,
    }
)

df_maq_info_prod_cad_cleaned.dtypes

fabrica                    object
linha                       int32
maquina_id               category
turno                    category
total_ciclos              float64
total_produzido           float64
data_registro      datetime64[ns]
hora_registro              object
dtype: object

In [ ]:
import sqlite3
import pandas as pd

# Cria uma conexão com o banco de dados SQLite (isso criará o arquivo se ele não existir)
conn = sqlite3.connect("..//assets//db_for_historic.db")

# Escreve um DataFrame do pandas no banco de dados
df_maq_info_cadastro.to_sql("maq_stopped", conn, if_exists="append", index=False)
df_maq_info_prod_cad_cleaned.to_sql(
    "info_production_cleaned", conn, if_exists="append", index=False
)
df_working_minutes.to_sql("time_working", conn, if_exists="append", index=False)

# Lê os dados de volta em um DataFrame
df = pd.read_sql_query("SELECT * FROM maq_stopped", conn)

# Fecha a conexão com o banco de dados
conn.close()

df

,fabrica,linha,maquina_id,turno,status,data_registro,hora_registro,motivo,equipamento,problema,causa,os_numero,operador_id,data_registro_ihm,hora_registro_ihm,data_hora,data_hora_final,tempo
0,1,8,TMF001,NOT,parada,2024-01-01 00:00:00,00:00:21.000000,Parada Programada,None,Feriado,None,None,None,2024-01-01 00:00:00,00:00:21.000000,2024-01-01 00:00:21,2024-01-01 08:02:22,480
1,1,8,TMF001,MAT,parada,2024-01-01 00:00:00,08:02:22.000000,Parada Programada,None,Feriado,None,None,None,2024-01-01 00:00:00,08:02:22.000000,2024-01-01 08:02:22,2024-01-01 16:02:24,480
2,1,8,TMF001,VES,parada,2024-01-01 00:00:00,16:02:24.000000,Parada Programada,None,Feriado,None,None,None,2024-01-01 00:00:00,16:02:24.000000,2024-01-01 16:02:24,2024-01-02 00:02:25,480
3,1,8,TMF001,NOT,parada,2024-01-02 00:00:00,00:02:25.000000,Parada Programada,None,None,None,None,None,2024-01-02 00:00:00,00:02:25.000000,2024-01-02 00:02:25,2024-01-02 08:02:27,480
4,1,8,TMF001,MAT,parada,2024-01-02 00:00:00,08:02:27.000000,Parada Programada,None,None,None,None,None,2024-01-02 00:00:00,08:02:27.000000,2024-01-02 08:02:27,2024-01-02 16:02:28,480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11938,1,3,TMF015,MAT,parada,2024-03-30 00:00:00,08:01:18.000000,Parada Programada,None,Sábado,None,None,None,2024-03-30 00:00:00,08:01:18.000000,2024-03-30 08:01:18,2024-03-30 16:00:24,480
11939,1,3,TMF015,VES,parada,2024-03-30 00:00:00,16:00:24.000000,Parada Programada,None,Sábado,None,None,None,2024-03-30 00:00:00,16:00:24.000000,2024-03-30 16:00:24,2024-03-31 00:02:16,480
11940,1,3,TMF015,NOT,parada,2024-03-31 00:00:00,00:02:16.000000,Parada Programada,None,Domingo,None,None,000441,2024-03-31 00:00:00,00:02:16.000000,2024-03-31 00:02:16,2024-03-31 08:02:17,480
11941,1,3,TMF015,MAT,parada,2024-03-31 00:00:00,08:02:17.000000,Parada Programada,None,Domingo,None,None,None,2024-03-31 00:00:00,08:02:17.000000,2024-03-31 08:02:17,2024-03-31 16:00:19,480


## Dados de Saída


In [ ]:
print(f"Máquina Info -> Linhas x Colunas: {df_maq_info_occ_combined.shape}")
print("----------------------------------------")
print(
    f"Máquina Info -> Memória: {df_maq_info_occ_combined.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB"
)
print("----------------------------------------")
print(f"Máquina Info -> Tipos:\n{df_maq_info_occ_combined.dtypes}")

Máquina Info -> Linhas x Colunas: (3724, 19)
----------------------------------------
Máquina Info -> Memória: 0.98 MB
----------------------------------------
Máquina Info -> Tipos:
maquina_id                        category
linha                             category
fabrica                           category
turno                             category
data_hora_registro          datetime64[ns]
tempo_registro_min                   int32
data_hora_final             datetime64[ns]
status                              object
motivo_id                          float64
data_hora_registro_occ      datetime64[ns]
motivo_nome                         object
problema                            object
solucao                             object
usuario_id_occ                    category
contagem_total_ciclos                int32
contagem_total_produzido             int32
sabado                               int32
domingo                              int32
feriado                              int32


In [ ]:
print(f"Máquina Produção -> Linhas x Colunas: {df_maq_info_production_cleaned.shape}")
print("----------------------------------------")
print(
    f"Máquina Produção -> Memória: {df_maq_info_production_cleaned.memory_usage(deep=True).sum() / 1024 ** 2:.2f} MB"
)
print("----------------------------------------")
print(f"Máquina Produção -> Tipos:\n{df_maq_info_production_cleaned.dtypes}")

Máquina Produção -> Linhas x Colunas: (305896, 10)
----------------------------------------
Máquina Produção -> Memória: 52.13 MB
----------------------------------------
Máquina Produção -> Tipos:
fabrica                    object
linha                    category
maquina_id               category
turno                    category
status                     object
total_ciclos                int32
total_produzido             int32
data_registro      datetime64[ns]
hora_registro              object
rn                          int64
dtype: object


# Análise de Dados - Eficiência, Performance, Reparos


## Análise de Dados


In [ ]:
# service/times_data.py
# cSpell: disable=invalid-name
class TimesData:
    def __init__(self):
        # Dicionário com os descontos de parada para Eficiência

        self.desc_eff = {
            3: 60,
            5: 10,
            10: 15,
            11: 35,
            15: 60,
            17: 15,
        }

        # Dicionário com os descontos de parada para Performance

        self.desc_perf = {
            3: 60,
            5: 10,
            10: 15,
            15: 60,
            17: 15,
        }

        # Dicionário com os descontos de parada para Reparos

        self.desc_rep = {11: 35}

        # Lista com os motivos de parada que não são considerados para Performance

        self.not_af_perf = [7, 8, 11, 12, 13, 16]

        # Lista com os motivos de parada que são considerados para Reparos

        self.af_rep = [7, 8, 11]

        # Lista de Motivos que não afetam a eficiência
        self.not_af_eff = [12]

    def get_times_discount(self, info: pd.DataFrame, desc_pcp: dict[int, int]) -> pd.DataFrame:
        """
        Função para calcular os descontos de parada

        Args:
            info (pd.DataFrame): DataFrame com os dados de parada
            desc_pcp (dict[int, int]): Dicionário com os descontos de parada

        Returns:
            pd.DataFrame: DataFrame com os descontos de parada


        Exemplo:
            >>> from app.service.get_times_data import GetTimesData
            >>> import pandas as pd
            >>> get_times_data = GetTimesData()
            >>> df_times_desc = pd.dataframe()
            >>> df_result = get_times_data.get_times_discount(df_times_desc, desc_pcp)
        """

        info_stops = info.copy()

        # Adicionar coluna com descontos de parada
        info_stops["desconto_min"] = info_stops["motivo_id"].map(desc_pcp)

        # Se houver desconto, subtrair do tempo de parada e arredondar para baixo, em uma nova coluna chamada excedente
        info_stops["excedente"] = (
            info_stops["tempo_registro_min"] - info_stops["desconto_min"]
        ).clip(lower=0)

        # Se o desconto for maior que o tempo de parada, o desconto deve ser igual ao tempo de parada
        info_stops.loc[
            info_stops["desconto_min"] > info_stops["tempo_registro_min"],
            "desconto_min",
        ] = info_stops["tempo_registro_min"]

        # Criar coluna data_registro para agrupar por dia
        info_stops["data_registro"] = info_stops["data_hora_registro"].dt.date
        # Ordenar por maquina_id, data_hora_registro, turno
        info_stops.sort_values(by=["maquina_id", "data_hora_registro", "turno"], inplace=True)

        return info_stops

    def get_elapsed_time(self, turno):
        """
        Método para calcular o tempo decorrido no turno atual.

        Este método recebe o turno atual e retorna o tempo decorrido em minutos.

        Args:
            turno (str): Turno atual

        Returns:
            float: Tempo decorrido em minutos


        Exemplo:
            >>> from app.service.get_times_data import GetTimesData
            >>> import pandas as pd
            >>> get_times_data = GetTimesData()
            >>> turno = 'MAT'
            >>> tempo_decorrido = get_times_data.get_elapsed_time(turno)
        """

        now = datetime.now()

        if turno == "MAT" and 8 <= now.hour < 16:
            shift_start = now.replace(hour=8, minute=0, second=0, microsecond=0)

        elif turno == "VES" and 16 <= now.hour < 24:
            shift_start = now.replace(hour=16, minute=0, second=0, microsecond=0)

        elif turno == "NOT" and (now.hour < 8 or now.hour >= 24):
            shift_start = now.replace(hour=0, minute=0, second=0, microsecond=0)

        else:
            return 480  # retorna o tempo padrão se não estiver no turno atual

        elapsed_time = now - shift_start

        return elapsed_time.total_seconds() / 60  # retorna o tempo decorrido em minutos

    def get_eff_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de eficiência.
        Este método recebe dois DataFrames, um contendo informações de tempo de eficiência e
        desconto e outro contendo informações de produção,
        e retorna um DataFrame com informações de eficiência.


        Parâmetros:

        df_info (pd.DataFrame): DataFrame contendo informações de maquina
        df_prod (pd.DataFrame): DataFrame contendo informações de produção.


        Retorna:

        pd.DataFrame: DataFrame com informações de eficiência.


        Exemplo de uso:
        ```
        times_data = TimesData()
        df_eff_times_desc = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_eff_data(df_info, df_prod)
        ```
        """

        df_eff_times_desc = self.get_times_discount(df_info, self.desc_eff)
        df_prod_total = df_prod.copy()
        ciclo_ideal = 10.6

        # Descartar colunas desnecessárias de df_prod -> 'contagem_total_ciclos', 'usuario_id_maq_cadastro', 'data_hora_registro'
        df_prod_total.drop(columns=["total_ciclos"], inplace=True)

        # Se o motivo id não afeta a eficiência, desconto_min deve ser igual ao tempo_registro_min
        df_eff_times_desc.loc[
            df_eff_times_desc["motivo_id"].isin(self.not_af_eff), "desconto_min"
        ] = df_eff_times_desc["tempo_registro_min"]

        # Agrupar por maquina_id, data_registro e turno e o desconto
        df_eff_times_desc = (
            df_eff_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                }
            )
            .reset_index()
        )

        # Remover onde a linha for 0
        df_eff_times_desc = df_eff_times_desc[df_eff_times_desc["linha"] != 0]

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_eff_times_desc["data_registro"] = pd.to_datetime(
            df_eff_times_desc["data_registro"]
        ).dt.date

        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date

        # Fazer merge com df_prod_total
        df_eff_times_desc = pd.merge(
            df_prod_total,
            df_eff_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desc_min para 0 quando for nulo
        df_eff_times_desc.loc[df_eff_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Criar coluna com tempo esperado de produção
        df_eff_times_desc["tempo_esperado_min"] = df_eff_times_desc.apply(
            lambda row: (
                np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
                if row["data_registro"] == datetime.now().date()
                else 480 - row["desconto_min"]
            ),
            axis=1,
        )

        # Produção esperada por turno
        df_eff_times_desc["producao_esperada"] = (
            df_eff_times_desc["tempo_esperado_min"] * ciclo_ideal
        ) * 2

        # Calcular a eficiência
        df_eff_times_desc["eficiencia"] = (
            df_eff_times_desc["total_produzido"] / df_eff_times_desc["producao_esperada"]
        )

        # Ordenar pela linha e data_registro
        df_eff_times_desc = df_eff_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_eff_times_desc = df_eff_times_desc[df_eff_times_desc["linha"] != 0]

        # Se eficiencia for nula, substituir por 0
        df_eff_times_desc.loc[df_eff_times_desc["eficiencia"].isnull(), "eficiencia"] = 0

        # Se a produção esperada for 0 e a eficiência for 0, substituir por np.nan
        df_eff_times_desc.loc[
            (df_eff_times_desc["producao_esperada"] == 0),
            "eficiencia",
        ] = np.nan

        # Ajustar o index
        df_eff_times_desc.reset_index(drop=True, inplace=True)

        return df_eff_times_desc

    def get_perf_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de performance.

        Este método recebe dois DataFrames, um contendo informações de máquina e
        outro contendo informações de produção,
        e retorna um DataFrame com informações de performance.

        Args:
            df_info (pd.DataFrame): DataFrame contendo informações de maquina
            df_prod (pd.DataFrame): DataFrame contendo informações de produção.

        Returns:
            pd.DataFrame: DataFrame com informações de performance.

        Exemplo de uso:
        ```
        times_data = TimesData()
        df_info = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_perf_data(df_info, df_prod)
        ```
        """

        df_info = df_info.copy()
        df_perf_times_desc = self.get_times_discount(df_info, self.desc_perf)
        df_prod_total = df_prod.copy()

        # Descartar colunas desnecessárias de df_prod
        df_prod_total.drop(
            columns=[
                "total_ciclos",
                "total_produzido",
            ],
            inplace=True,
        )

        # Conseguir apenas as datas/turno se o motivo_id = 12 e o tempo de registro for maior que 480
        datas_programadas = df_perf_times_desc[
            (df_perf_times_desc["motivo_id"] == 12)
            & (df_perf_times_desc["tempo_registro_min"] == 480)
        ][["linha", "data_registro", "turno"]]

        # Remover as linhas que não afetam a performance
        df_perf_times_desc = df_perf_times_desc[
            ~df_perf_times_desc["motivo_id"].isin(self.not_af_perf)
        ]

        # Criar coluna 'afeta' para identificar as paradas que afetam a performance
        df_perf_times_desc["afeta"] = df_perf_times_desc["excedente"]

        # Se desconto for nulo, substituir afeta pelo valor de tempo_registro_min
        df_perf_times_desc.loc[df_perf_times_desc["desconto_min"].isnull(), "afeta"] = (
            df_perf_times_desc["tempo_registro_min"]
        )

        # Agrupar por maquina_id, data_registro e turno e somar o tempo de
        # desconto e o afeta
        df_perf_times_desc = (
            df_perf_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                    "afeta": "sum",
                }
            )
            .reset_index()
        )

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_perf_times_desc["data_registro"] = pd.to_datetime(
            df_perf_times_desc["data_registro"]
        ).dt.date

        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date
        # Fazer merge com df_prod_total

        df_perf_times_desc = pd.merge(
            df_prod_total,
            df_perf_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desconto_min para 0 quando for nulo
        df_perf_times_desc.loc[df_perf_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Ajustar afeta para 0 quando for nulo
        df_perf_times_desc.loc[df_perf_times_desc["afeta"].isnull(), "afeta"] = 0

        # Criar coluna com tempo esperado de produção
        df_perf_times_desc["tempo_esperado_min"] = df_perf_times_desc.apply(
            lambda row: (
                np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
                if row["data_registro"] == datetime.now().date()
                else 480 - row["desconto_min"]
            ),
            axis=1,
        )

        # Calcular a performance
        df_perf_times_desc["performance"] = (
            df_perf_times_desc["afeta"] / df_perf_times_desc["tempo_esperado_min"]
        )

        # Ordenar pela linha e data_registro
        df_perf_times_desc = df_perf_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_perf_times_desc = df_perf_times_desc[df_perf_times_desc["linha"] != 0]

        # Se a data e o turno coincidem com datas_programadas, a performance é np.nan
        # Converter 'data_registro' para datetime
        df_perf_times_desc["data_registro"] = pd.to_datetime(df_perf_times_desc["data_registro"])
        datas_programadas["data_registro"] = pd.to_datetime(datas_programadas["data_registro"])

        # Criar uma chave única em ambos os DataFrames
        df_perf_times_desc["key"] = (
            df_perf_times_desc["linha"].astype(str)
            + df_perf_times_desc["data_registro"].dt.strftime("%Y-%m-%d")
            + df_perf_times_desc["turno"].astype(str)
        )
        datas_programadas["key"] = (
            datas_programadas["linha"].astype(str)
            + datas_programadas["data_registro"].dt.strftime("%Y-%m-%d")
            + datas_programadas["turno"].astype(str)
        )

        # Se a chave coincide com datas_programadas, a performance é np.nan
        df_perf_times_desc.loc[
            df_perf_times_desc["key"].isin(datas_programadas["key"]),
            "performance",
        ] = np.nan

        # Remover a coluna 'key'
        df_perf_times_desc.drop(columns=["key"], inplace=True)
        datas_programadas.drop(columns=["key"], inplace=True)

        # Ajustar o index
        df_perf_times_desc.reset_index(drop=True, inplace=True)

        return df_perf_times_desc

    def get_repair_data(self, df_info: pd.DataFrame, df_prod: pd.DataFrame) -> pd.DataFrame:
        """
        Método para calcular os dados de reparo.

        Este método recebe dois DataFrames, um contendo informações de máquina e
        e outro contendo informações de produção,
        e retorna um DataFrame com informações de Reparo.


        ### Parâmetros:
        df_info (pd.DataFrame): DataFrame contendo informações de maquina

        df_prod (pd.DataFrame): DataFrame contendo informações de produção.

        ### Retorna:

        pd.DataFrame: DataFrame com informações de performance.

        ### Exemplo de uso:
        ```
        times_data = TimesData()
        df_info = pd.dataframe()
        df_prod = pd.dataframe()
        df_result = times_data.get_repair_data(df_info, df_prod)
        ```
        """

        df_info = df_info.copy()

        df_rep_times_desc = self.get_times_discount(df_info, self.desc_rep)

        df_prod_total = df_prod.copy()

        # Descartar colunas desnecessárias de df_prod
        df_prod_total.drop(
            columns=[
                "total_ciclos",
                "total_produzido",
            ],
            inplace=True,
        )

        # Conseguir apenas as datas que o motivo_id = 12 e o tempo de registro for maior que 480
        datas_programadas = df_rep_times_desc[
            (df_rep_times_desc["motivo_id"] == 12)
            & (df_rep_times_desc["tempo_registro_min"] == 480)
        ][["linha", "data_registro", "turno"]]

        # Remover as linhas que não afetam o reparo
        df_rep_times_desc = df_rep_times_desc[df_rep_times_desc["motivo_id"].isin(self.af_rep)]

        # Criar coluna 'afeta' para identificar as paradas que afetam o reparo
        df_rep_times_desc["afeta"] = df_rep_times_desc["excedente"]

        # Se desconto for nulo, substituir afeta pelo valor de tempo_registro_min
        df_rep_times_desc.loc[df_rep_times_desc["desconto_min"].isnull(), "afeta"] = (
            df_rep_times_desc["tempo_registro_min"]
        )

        # Agrupar por maquina_id, data_registro e turno e somar o tempo de
        # desconto e o afeta
        df_rep_times_desc = (
            df_rep_times_desc.groupby(
                ["maquina_id", "linha", "data_registro", "turno"], observed=False
            )
            .agg(
                {
                    "desconto_min": "sum",
                    "afeta": "sum",
                }
            )
            .reset_index()
        )

        # Garantir que a coluna data_registro é datetime em ambos os dataframes
        df_rep_times_desc["data_registro"] = pd.to_datetime(
            df_rep_times_desc["data_registro"]
        ).dt.date
        df_prod_total["data_registro"] = pd.to_datetime(df_prod_total["data_registro"]).dt.date

        # Fazer merge com df_prod_total
        df_rep_times_desc = pd.merge(
            df_prod_total,
            df_rep_times_desc,
            on=["maquina_id", "linha", "turno", "data_registro"],
            how="left",
        )

        # Ajustar desconto_min para 0 quando for nulo
        df_rep_times_desc.loc[df_rep_times_desc["desconto_min"].isnull(), "desconto_min"] = 0

        # Ajustar afeta para 0 quando for nulo
        df_rep_times_desc.loc[df_rep_times_desc["afeta"].isnull(), "afeta"] = 0

        # Criar coluna com tempo esperado de produção
        df_rep_times_desc["tempo_esperado_min"] = df_rep_times_desc.apply(
            lambda row: (
                np.floor(self.get_elapsed_time(row["turno"]) - row["desconto_min"])
                if row["data_registro"] == datetime.now().date()
                else 480 - row["desconto_min"]
            ),
            axis=1,
        )

        # Calcular o reparo
        df_rep_times_desc["reparo"] = (
            df_rep_times_desc["afeta"] / df_rep_times_desc["tempo_esperado_min"]
        )

        # Ordenar pela linha e data_registro
        df_rep_times_desc = df_rep_times_desc.sort_values(
            by=["linha", "data_registro"], ascending=True
        )

        # Remover as linhas onde a linha é 0
        df_rep_times_desc = df_rep_times_desc[df_rep_times_desc["linha"] != 0]

        # Se a data e o turno coincidem com datas_programadas, a performance é np.nan
        # Converter 'data_registro' para datetime
        df_rep_times_desc["data_registro"] = pd.to_datetime(df_rep_times_desc["data_registro"])
        datas_programadas["data_registro"] = pd.to_datetime(datas_programadas["data_registro"])

        # Criar uma chave única em ambos os DataFrames
        df_rep_times_desc["key"] = (
            df_rep_times_desc["linha"].astype(str)
            + df_rep_times_desc["data_registro"].dt.strftime("%Y-%m-%d")
            + df_rep_times_desc["turno"].astype(str)
        )
        datas_programadas["key"] = (
            datas_programadas["linha"].astype(str)
            + datas_programadas["data_registro"].dt.strftime("%Y-%m-%d")
            + datas_programadas["turno"].astype(str)
        )

        # Se a chave coincide com datas_programadas, a performance é np.nan
        df_rep_times_desc.loc[
            df_rep_times_desc["key"].isin(datas_programadas["key"]),
            "reparo",
        ] = np.nan

        # Remover a coluna 'key'
        df_rep_times_desc.drop(columns=["key"], inplace=True)
        datas_programadas.drop(columns=["key"], inplace=True)

        # Ajustar o index
        df_rep_times_desc.reset_index(drop=True, inplace=True)

        return df_rep_times_desc


times_data = TimesData()

## Saída de Dados


### Eficiência


In [ ]:
df_eff = times_data.get_eff_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_eff.head(28)

,fabrica,linha,maquina_id,turno,status,total_produzido,data_registro,hora_registro,rn,desconto_min,tempo_esperado_min,producao_esperada,eficiencia
0,1,1,TMF005,NOT,true,8858,2024-03-01,07:59:24.736666,1,0.0,480.0,10176.0,0.870480
1,1,1,TMF005,NOT,true,8812,2024-03-01,07:57:24.730000,2,0.0,480.0,10176.0,0.865959
2,1,1,TMF005,NOT,true,8768,2024-03-01,07:55:24.723333,3,0.0,480.0,10176.0,0.861635
3,1,1,TMF005,NOT,true,8726,2024-03-01,07:53:24.716666,4,0.0,480.0,10176.0,0.857508
4,1,1,TMF005,NOT,true,8684,2024-03-01,07:51:24.710000,5,0.0,480.0,10176.0,0.853381
5,1,1,TMF005,NOT,true,8640,2024-03-01,07:49:24.703333,6,0.0,480.0,10176.0,0.849057
6,1,1,TMF005,NOT,true,8598,2024-03-01,07:47:24.696666,7,0.0,480.0,10176.0,0.844929
7,1,1,TMF005,NOT,true,8556,2024-03-01,07:45:24.693333,8,0.0,480.0,10176.0,0.840802
8,1,1,TMF005,NOT,true,8514,2024-03-01,07:43:24.686666,9,0.0,480.0,10176.0,0.836675
9,1,1,TMF005,NOT,true,8472,2024-03-01,07:41:24.680000,10,0.0,480.0,10176.0,0.832547


### Performance


In [ ]:
df_perf = times_data.get_perf_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_perf.head(28)

,fabrica,linha,maquina_id,turno,status,data_registro,hora_registro,rn,desconto_min,afeta,tempo_esperado_min,performance
0,1,1,TMF005,NOT,true,2024-03-01,07:59:24.736666,1,0.0,84.0,480.0,0.175
1,1,1,TMF005,NOT,true,2024-03-01,07:57:24.730000,2,0.0,84.0,480.0,0.175
2,1,1,TMF005,NOT,true,2024-03-01,07:55:24.723333,3,0.0,84.0,480.0,0.175
3,1,1,TMF005,NOT,true,2024-03-01,07:53:24.716666,4,0.0,84.0,480.0,0.175
4,1,1,TMF005,NOT,true,2024-03-01,07:51:24.710000,5,0.0,84.0,480.0,0.175
5,1,1,TMF005,NOT,true,2024-03-01,07:49:24.703333,6,0.0,84.0,480.0,0.175
6,1,1,TMF005,NOT,true,2024-03-01,07:47:24.696666,7,0.0,84.0,480.0,0.175
7,1,1,TMF005,NOT,true,2024-03-01,07:45:24.693333,8,0.0,84.0,480.0,0.175
8,1,1,TMF005,NOT,true,2024-03-01,07:43:24.686666,9,0.0,84.0,480.0,0.175
9,1,1,TMF005,NOT,true,2024-03-01,07:41:24.680000,10,0.0,84.0,480.0,0.175


### Reparos


In [ ]:
df_reparos = times_data.get_repair_data(
    df_maq_info_occ_combined.copy(), df_maq_info_production_cleaned.copy()
)
df_reparos.head(28)

,fabrica,linha,maquina_id,turno,status,data_registro,hora_registro,rn,desconto_min,afeta,tempo_esperado_min,reparo
0,1,1,TMF005,NOT,true,2024-03-01,07:59:24.736666,1,0.0,0.0,480.0,0.0
1,1,1,TMF005,NOT,true,2024-03-01,07:57:24.730000,2,0.0,0.0,480.0,0.0
2,1,1,TMF005,NOT,true,2024-03-01,07:55:24.723333,3,0.0,0.0,480.0,0.0
3,1,1,TMF005,NOT,true,2024-03-01,07:53:24.716666,4,0.0,0.0,480.0,0.0
4,1,1,TMF005,NOT,true,2024-03-01,07:51:24.710000,5,0.0,0.0,480.0,0.0
5,1,1,TMF005,NOT,true,2024-03-01,07:49:24.703333,6,0.0,0.0,480.0,0.0
6,1,1,TMF005,NOT,true,2024-03-01,07:47:24.696666,7,0.0,0.0,480.0,0.0
7,1,1,TMF005,NOT,true,2024-03-01,07:45:24.693333,8,0.0,0.0,480.0,0.0
8,1,1,TMF005,NOT,true,2024-03-01,07:43:24.686666,9,0.0,0.0,480.0,0.0
9,1,1,TMF005,NOT,true,2024-03-01,07:41:24.680000,10,0.0,0.0,480.0,0.0


# Componentes


## Auxiliares - Preparação de DF's


### Class DFIndicators - DF's auxiliares para a preparação dos dados.


In [ ]:
class DFIndicators:
    def __init__(self, df_info_ind: pd.DataFrame, df_prod_ind: pd.DataFrame):
        self.times_data = TimesData()
        self.df_info = df_info_ind
        self.df_prod = df_prod_ind
        self.indicator_functions = {
            IndicatorType.EFFICIENCY: self.times_data.get_eff_data,
            IndicatorType.PERFORMANCE: self.times_data.get_perf_data,
            IndicatorType.REPAIR: self.times_data.get_repair_data,
        }
        self.indicator_desc = {
            IndicatorType.EFFICIENCY: self.times_data.desc_eff,
            IndicatorType.PERFORMANCE: self.times_data.desc_perf,
            IndicatorType.REPAIR: self.times_data.desc_rep,
        }

    # ---------------- df Heatmap ---------------- #

    def get_heatmap_data(
        self, indicator: IndicatorType, turn: str = None, main: bool = False
    ) -> pd.DataFrame:

        # Cria um dataframe vazio
        dataframe = pd.DataFrame()

        # Verifica se o indicador está no dicionário, se sim, chama a função do indicador
        if indicator in self.indicator_functions:
            dataframe = self.indicator_functions[indicator](self.df_info, self.df_prod)

        # Se o turno for diferente de nulo, filtra o dataframe
        if turn:
            dataframe = dataframe[dataframe["turno"] == turn]

        # Converter 'data_registro' para datetime
        dataframe["data_registro"] = pd.to_datetime(dataframe["data_registro"])

        # Criar coluna 'data_turno' para agrupar por dia e turno
        dataframe["data_turno"] = dataframe["data_registro"].dt.strftime("%Y-%m-%d")

        group_col = ["data_turno", "linha"] if not main else ["data_turno", "turno"]

        # Agrupar por data_turno e turno e calcular a média do indicador
        df_grouped = (
            dataframe.groupby(group_col, observed=False)[indicator.value].mean().reset_index()
        )

        # ------------ dataframe com datas possíveis ------------ #
        # Obter a data de início e fim do mês
        today = datetime.now()
        start_date = today.replace(day=1).strftime("%Y-%m-%d")
        end_date = (today.replace(month=today.month % 12 + 1, day=1) - timedelta(days=1)).strftime(
            "%Y-%m-%d"
        )

        # Criar um dataframe com as datas possíveis
        all_dates = pd.date_range(start=start_date, end=end_date).strftime("%Y-%m-%d")
        all_lines = dataframe["linha"].unique() if not main else ["MAT", "VES", "NOT"]
        all_dates_lines = pd.DataFrame(list(product(all_dates, all_lines)), columns=group_col)

        # Merge com o dataframe agrupado
        df_grouped = pd.merge(all_dates_lines, df_grouped, on=group_col, how="left")

        # Se a data é futura, o indicador é np.nan
        df_grouped.loc[df_grouped["data_turno"] > today.strftime("%Y-%m-%d"), indicator.value] = (
            np.nan
        )

        # Pivotar o dataframe
        if main:
            df_pivot = df_grouped.pivot(index="turno", columns="data_turno", values=indicator.value)
            df_pivot = df_pivot.reindex(["VES", "MAT", "NOT"])
        else:
            df_grouped = df_grouped.sort_values(by=["linha", "data_turno"], ascending=True)
            df_pivot = df_grouped.pivot(index="linha", columns="data_turno", values=indicator.value)

        # Remover a linha 0
        df_pivot = df_pivot[df_pivot.index != 0]

        return df_pivot

    # ---------------- df Time Lost ---------------- #

    def get_time_lost_data(
        self, indicator: IndicatorType, turn: str = "TOT", working_minutes: pd.DataFrame = None
    ) -> pd.DataFrame:
        # Mapear função pelo indicador
        indicator_function = self.indicator_desc[indicator]

        # Dataframe com tempos de desconto
        df_times_desc = self.times_data.get_times_discount(self.df_info, indicator_function)

        # Unir com o dataframe working_minutes
        if working_minutes is not None:
            df_times_desc = pd.concat(
                [df_times_desc, working_minutes], ignore_index=True, sort=False
            )

        # Filtrar por turno
        if turn != "TOT":
            df_times_desc = df_times_desc[df_times_desc["turno"] == turn]

        # Se coluna "excedente" for nula, substituir pelo valor de "tempo_registro_min"
        df_times_desc.loc[df_times_desc["excedente"].isnull(), "excedente"] = df_times_desc[
            "tempo_registro_min"
        ]

        # Se o 'problema' for uma string 'NONE' substituir por 'Não Informado'
        df_times_desc.loc[df_times_desc["problema"] == "NONE", "problema"] = "Não Informado"

        # motivo_nome é '5 min ou menos' se motivo_id é nulo e excedente é menor ou igual a 5
        mask = (df_times_desc["motivo_id"].isnull()) & (df_times_desc["excedente"] <= 5)
        df_times_desc["motivo_nome"] = np.where(
            mask, "5 min ou menos", df_times_desc["motivo_nome"]
        )
        df_times_desc["problema"] = np.where(mask, "5 min ou menos", df_times_desc["problema"])

        # Preencher motivo_nome com 'Motivo não apontado' se motivo_id for nulo
        df_times_desc["motivo_nome"] = df_times_desc["motivo_nome"].fillna("Motivo não apontado")

        # Corrigir parada programada
        mask = (df_times_desc["motivo_id"] == 12) & (
            df_times_desc["problema"] == "Parada programada"
        )
        df_times_desc["problema"] = np.where(mask, "Parada Programada", df_times_desc["problema"])

        # Ajustar data_hora_final para datetime
        df_times_desc["data_hora_final"] = pd.to_datetime(df_times_desc["data_hora_final"])

        # Ordenar por linha e data_hora_registro
        df_times_desc = df_times_desc.sort_values(by=["linha", "data_hora_registro"])

        # Ajustar o index
        df_times_desc.reset_index(drop=True, inplace=True)

        return df_times_desc


df_to_indicators = DFIndicators(df_maq_info_occ_combined, df_maq_info_production_cleaned)

### Saída de dados - DF's auxiliares


In [ ]:
df_to_indicators.get_heatmap_data(IndicatorType.EFFICIENCY, main=True)

data_turno,2024-05-01,2024-05-02,2024-05-03,2024-05-04,2024-05-05,2024-05-06,2024-05-07,2024-05-08,2024-05-09,2024-05-10,...,2024-05-22,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31
turno,,,,,,,,,,,,,,,,,,,,,
VES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_to_indicators.get_heatmap_data(IndicatorType.PERFORMANCE, "MAT")

data_turno,2024-05-01,2024-05-02,2024-05-03,2024-05-04,2024-05-05,2024-05-06,2024-05-07,2024-05-08,2024-05-09,2024-05-10,...,2024-05-22,2024-05-23,2024-05-24,2024-05-25,2024-05-26,2024-05-27,2024-05-28,2024-05-29,2024-05-30,2024-05-31
linha,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_to_indicators.get_time_lost_data(IndicatorType.EFFICIENCY, "MAT")

,maquina_id,linha,fabrica,turno,data_hora_registro,tempo_registro_min,data_hora_final,status,motivo_id,data_hora_registro_occ,...,solucao,usuario_id_occ,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado,desconto_min,excedente,data_registro
0,TMF005,1,1,MAT,2024-03-01 08:23:24,10,2024-03-01 08:33:24,parada,NaN,NaT,...,,NaN,484,480,0,0,0,NaN,10.0,2024-03-01
1,TMF005,1,1,MAT,2024-03-01 09:23:24,2,2024-03-01 09:25:24,parada,NaN,NaT,...,,NaN,1552,1548,0,0,0,NaN,2.0,2024-03-01
2,TMF005,1,1,MAT,2024-03-01 10:55:25,132,2024-03-01 13:07:25,parada,NaN,NaT,...,,NaN,3442,3424,0,0,0,NaN,132.0,2024-03-01
3,TMF005,1,1,MAT,2024-03-01 14:31:25,2,2024-03-01 14:33:25,parada,NaN,NaT,...,,NaN,5206,5188,0,0,0,NaN,2.0,2024-03-01
4,TMF005,1,1,MAT,2024-03-01 14:45:25,2,2024-03-01 14:47:25,parada,NaN,NaT,...,,NaN,5488,5470,0,0,0,NaN,2.0,2024-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,TMF009,14,2,MAT,2024-03-27 08:02:18,480,2024-03-27 16:02:24,parada,8.0,2024-03-27 08:11:04,...,,000807,0,0,0,0,0,NaN,480.0,2024-03-27
1396,TMF009,14,2,MAT,2024-03-28 08:02:27,480,2024-03-28 16:02:29,parada,12.0,2024-03-28 08:16:01,...,,000807,0,0,0,0,0,NaN,480.0,2024-03-28
1397,TMF009,14,2,MAT,2024-03-29 08:02:32,480,2024-03-29 16:02:33,parada,12.0,NaT,...,,NaN,0,0,0,0,1,NaN,480.0,2024-03-29
1398,TMF009,14,2,MAT,2024-03-30 08:02:36,480,2024-03-30 16:01:38,parada,12.0,NaT,...,,NaN,0,0,1,0,0,NaN,480.0,2024-03-30


In [ ]:
df_to_indicators.get_time_lost_data(IndicatorType.PERFORMANCE, "MAT", df_time_working)

,maquina_id,linha,fabrica,turno,data_hora_registro,tempo_registro_min,data_hora_final,status,motivo_id,data_hora_registro_occ,...,solucao,usuario_id_occ,contagem_total_ciclos,contagem_total_produzido,sabado,domingo,feriado,desconto_min,excedente,data_registro
0,TMF005,1,1,MAT,2024-03-01 08:23:24,10,2024-03-01 08:33:24,parada,NaN,NaT,...,,NaN,484.0,480.0,0.0,0.0,0.0,NaN,10.0,2024-03-01
1,TMF005,1,1,MAT,2024-03-01 09:23:24,2,2024-03-01 09:25:24,parada,NaN,NaT,...,,NaN,1552.0,1548.0,0.0,0.0,0.0,NaN,2.0,2024-03-01
2,TMF005,1,1,MAT,2024-03-01 10:55:25,132,2024-03-01 13:07:25,parada,NaN,NaT,...,,NaN,3442.0,3424.0,0.0,0.0,0.0,NaN,132.0,2024-03-01
3,TMF005,1,1,MAT,2024-03-01 14:31:25,2,2024-03-01 14:33:25,parada,NaN,NaT,...,,NaN,5206.0,5188.0,0.0,0.0,0.0,NaN,2.0,2024-03-01
4,TMF005,1,1,MAT,2024-03-01 14:45:25,2,2024-03-01 14:47:25,parada,NaN,NaT,...,,NaN,5488.0,5470.0,0.0,0.0,0.0,NaN,2.0,2024-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,TMF009,14,2,MAT,2024-03-27 08:02:18,480,2024-03-27 16:02:24,parada,8.0,2024-03-27 08:11:04,...,,000807,0.0,0.0,0.0,0.0,0.0,NaN,480.0,2024-03-27
1595,TMF009,14,2,MAT,2024-03-28 08:02:27,480,2024-03-28 16:02:29,parada,12.0,2024-03-28 08:16:01,...,,000807,0.0,0.0,0.0,0.0,0.0,NaN,480.0,2024-03-28
1596,TMF009,14,2,MAT,2024-03-29 08:02:32,480,2024-03-29 16:02:33,parada,12.0,NaT,...,,NaN,0.0,0.0,0.0,0.0,1.0,NaN,480.0,2024-03-29
1597,TMF009,14,2,MAT,2024-03-30 08:02:36,480,2024-03-30 16:01:38,parada,12.0,NaT,...,,NaN,0.0,0.0,1.0,0.0,0.0,NaN,480.0,2024-03-30


In [ ]:
df_production_per_hour = df_info.copy()

# Encontra o dia de hoje
today = datetime.now().date()

# Filtra os dados para o dia de hoje
df_production_per_hour = df_production_per_hour[df_production_per_hour["data_registro"] == today]

# Converta 'data_registro' para string antes de concatenar
df_production_per_hour["data_hora"] = (
    df_production_per_hour["data_registro"].astype(str)
    + " "
    + df_production_per_hour["hora_registro"].astype(str).str.split(".").str[0]
)

df_production_per_hour["data_hora"] = pd.to_datetime(df_production_per_hour["data_hora"])

# Defina 'linha' e 'data_hora' como índices
df_production_per_hour.set_index(["linha", "data_hora"], inplace=True)

# Agregue os dados por hora e aplique funções diferentes para diferentes colunas
df_resampled = (
    df_production_per_hour.groupby("linha")
    .resample("h", level="data_hora")
    .agg({"contagem_total_produzido": ["first", "last"]})
)

# Calcule a diferença entre o primeiro e o último registro de cada hora
df_resampled["total_produzido"] = (
    df_resampled["contagem_total_produzido"]["last"]
    - df_resampled["contagem_total_produzido"]["first"]
)

# Reiniciar o índice do DataFrame
df_resampled.reset_index(inplace=True)

# Selecionar apenas as colunas necessárias
df_final = df_resampled[["linha", "data_hora", "total_produzido"]]

# Dividir o total produzido por 10 para ter o número de caixas
df_final.loc[:, "total_produzido"] = np.floor(df_final["total_produzido"] / 10).astype(int)

# Pivotar a tabela
df_pivot = df_final.pivot(index="data_hora", columns="linha", values="total_produzido")

df_pivot

TypeError: <class 'pandas.core.indexes.multi.MultiIndex'>